In [47]:
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sortedcontainers import SortedList
import math

Tách thành tập train và test

In [48]:
df = pd.read_csv('../dataset/rating_final.csv')
df = shuffle(df)

cut_off_length = int(0.8 * len(df))
df_train = df[:cut_off_length]
df_test = df[cut_off_length:]

In [49]:
user_to_movie = {}
movie_to_user = {}
movie_user_rating = {}

for row in df_train.values:
    movie_id, user_id, rate = row
    if movie_id not in user_to_movie:
        user_to_movie[user_id] = [movie_id]
    else:
        user_to_movie[user_id].append(movie_id)
    
    if movie_id not in movie_to_user:
        movie_to_user[movie_id] = [user_id]
    else:
        movie_to_user[movie_id].append(user_id)

    movie_user_rating[(movie_id, user_id)] = rate

In [50]:
movie_user_rating_test = {}
for row in df_test.values:
    movie_id, user_id, rate = row

    movie_user_rating_test[(movie_id, user_id)] = rate

In [51]:
number_of_movie = int(len(movie_to_user.keys()))
number_of_user = int(len(user_to_movie.keys()))

print(f'Number of movie: {number_of_movie}')
print(f'Number of user: {number_of_user}')

Number of movie: 490
Number of user: 1000


In [52]:
k_neighbor = 20
limit_neighbor = 5
neighbors = {}
averages = {}

for movie_id_i in movie_to_user.keys():
    # get all user who rated for movie_id_i
    user_rated_movie_i = movie_to_user[movie_id_i]

    # get all rating of users gave for movie_id_i
    rating_user_for_movie_i = [movie_user_rating[(movie_id_i, user_id)] for user_id in user_rated_movie_i]

    # calculate avarage rating of all user gave for movie_id_i
    avg_rating_movie_i = np.mean(rating_user_for_movie_i)

    averages[movie_id] = avg_rating_movie_i

    sorted_list = SortedList()
    for movie_id_j in movie_to_user.keys():
        if movie_id_j == movie_id_i:
            continue
        
        # get all user who rated for movie_id_j
        user_rated_movie_j = movie_to_user[movie_id_j]

        common_users = list(set(user_rated_movie_i).intersection(set(user_rated_movie_j)))

        if len(common_users) < limit_neighbor:
            continue
        
        # get all rating of users gave for movie_id_j
        rating_user_for_movie_j = [movie_user_rating[(movie_id_j, user_id)] for user_id in user_rated_movie_j]

        # calculate avarage rating of all user gave for movie_id_j
        avg_rating_movie_j = np.mean(rating_user_for_movie_j)

        deviation = 0
        sigmoid_i = 0
        sigmoid_j = 0
        for user_id in common_users:
            dev_i = (movie_user_rating[(movie_id_i, user_id)] - avg_rating_movie_i)
            dev_j = (movie_user_rating[(movie_id_j, user_id)] - avg_rating_movie_j)
            deviation += dev_i * dev_j
            sigmoid_i += dev_i * dev_i
            sigmoid_j += dev_j * dev_j

        s_ij = deviation / (math.sqrt(sigmoid_i) * math.sqrt(sigmoid_j))
        sorted_list.add(s_ij)
        if len(sorted_list) > k_neighbor:
            del sorted_list[0]

    neighbors[movie_id_i] = sorted_list
print(neighbors)




C:\Users\dekub\AppData\Local\Temp\ipykernel_24020\1486776978.py:47: RuntimeWarning: invalid value encountered in double_scalars
  s_ij = deviation / (math.sqrt(sigmoid_i) * math.sqrt(sigmoid_j))


{'tt0034583': SortedList([0.6708638412462575, 0.6778260779094759, 0.6837044103419582, 0.6863461310040423, 0.6868916901771978, 0.6898297202111606, 0.7064118784149974, 0.7098830155636937, 0.739136421810468, 0.7413068774574192, 0.745597408532872, 0.7703296430456531, 0.7706971341185143, 0.8190152410680326, 0.8436411418954252, 0.8472864993984465, 0.8852407555119962, 0.8895253541537275, 0.9275653962424225, 0.9885407164193926]), 'tt0347149': SortedList([0.7879680707751967, 0.7970847727320788, 0.7975037006087777, 0.8034056931718038, 0.8226888558309017, 0.834898431296504, 0.8467762709445542, 0.847756400591001, 0.8952559965062987, 0.9037598935762619, 0.9109798710567623, 0.9217788461719337, 0.9250461349611099, 0.9281816986340269, 0.9536734254792716, 0.9914858860483589, 1.0, 1.0, 1.0000000000000002, 1.0000000000000002]), 'tt2024544': SortedList([0.661287067247112, 0.6713275895928081, 0.6877268850060779, 0.6920363943327343, 0.6965680457164123, 0.7057734942887347, 0.7122886257097679, 0.7150042512268